In [1]:
import xml.etree.ElementTree as ET
import pandas as pd
import gensim
import numpy as np
import nltk
from nltk.stem.snowball import SnowballStemmer 
from nltk.corpus import stopwords
import spacy
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D
from gensim.models import FastText

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.
C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Load data into pandas dataframe

In [2]:
class XML2DataFrame:
    def __init__(self, xml_path):
        xml_data = open(xml_path, encoding='utf-8')
        self.root = ET.XML(xml_data.read())[1]

    def parse_root(self, root):
        return [self.parse_element(child) for child in iter(root)]

    def parse_element(self, element, parsed=None):
        if parsed is None:
            parsed = dict()
        for key in element.keys():
            parsed[key] = element.attrib.get(key)
        if element.text:
            parsed[element.attrib["name"]] = None if element.text == "NULL" else element.text
        for child in list(element):
            self.parse_element(child, parsed)
        return parsed

    def process_data(self):
        structure_data = self.parse_root(self.root)
        return pd.DataFrame(structure_data)

In [3]:
train = XML2DataFrame("data/tkk_train_2016.xml").process_data().fillna(0)
train_bank = XML2DataFrame("data/bank_train_2016.xml").process_data().fillna(0)
test = XML2DataFrame("data/tkk_test_etalon.xml").process_data().fillna(0)
test_bank = XML2DataFrame("data/banks_test_etalon.xml").process_data().fillna(0)

## Load train and test, form object vector and labels vector

In [4]:
train_text = train['text'].values
test_text = test['text'].values
train_labels = train[["beeline", "komstar", "mts", "rostelecom", "skylink", "tele2"]].astype(int).sum(axis=1).values
test_labels = test[["beeline", "komstar", "mts", "rostelecom", "skylink", "tele2"]].astype(int).sum(axis=1).values

In [5]:
def norm_labels(labels):
    for i in range(len(labels)):
        if labels[i]>=1:
            labels[i] = 1
        if labels[i]<=-1:
            labels[i] = -1
    return labels

train_labels = norm_labels(train_labels)
test_labels = norm_labels(test_labels)

In [6]:
set(train_labels)

{-1, 0, 1}

In [7]:
train_bank_text = train_bank['text'].values
test_bank_text = test_bank['text'].values
train_bank_labels = train_bank[['alfabank','bankmoskvy','gazprom','raiffeisen','rshb','sberbank','uralsib','vtb']].astype(int).sum(axis=1).values
test_bank_labels = test_bank[['alfabank','bankmoskvy','gazprom','raiffeisen','rshb','sberbank','uralsib','vtb']].astype(int).sum(axis=1).values

In [8]:
train_bank_labels = norm_labels(train_bank_labels)
test_bank_labels = norm_labels(test_bank_labels)

## Preprocess text - tokenize, delete stop-words, stem

In [9]:
stop = stopwords.words('russian')
mystem = SnowballStemmer('russian')
def preprocess_text(text):
    tokens = nltk.tokenize.RegexpTokenizer(r'\w+').tokenize(text)
    tokens = [mystem.stem(token) for token in tokens if token not in stop]
    
    text = " ".join(tokens)
    
    return text

In [10]:
train_ready = [preprocess_text(text) for text in train_text]
test_ready = [preprocess_text(text) for text in test_text]
train_bank_ready = [preprocess_text(text) for text in train_bank_text]
test_bank_ready = [preprocess_text(text) for text in test_bank_text]

## Convolutional network
for ttk

In [11]:
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation


make data ready for network

In [12]:
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(train_ready)

x_train = tokenizer.texts_to_sequences(train_ready)
x_test = tokenizer.texts_to_sequences(test_ready)

X_train = pad_sequences(x_train, maxlen=100)
X_test = pad_sequences(x_test, maxlen=100)

word_index = tokenizer.word_index

In [13]:
X_train.shape

(8643, 100)

In [14]:
encoder = LabelBinarizer()
encoder.fit(train_labels)
y_train = encoder.transform(train_labels)
y_test = encoder.transform(test_labels)

In [15]:
y_train.shape

(8643, 3)

create and train model

In [16]:
model = Sequential()

model.add(Embedding(10000, 1000, input_length=100))
model.add(Dropout(0.2))
model.add(Conv1D(256, 5, activation='relu'))
model.add(MaxPooling1D(pool_size=4))
model.add(LSTM(100))
model.add(Dense(3, activation='sigmoid'))


model.compile(loss='binary_crossentropy', optimizer='adam',    metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=32, epochs=15)

Epoch 1/15


6528/8643 [=====================>........] - ETA: 617s - loss: 0.6932 - acc: 0.364 - ETA: 496s - loss: 0.6680 - acc: 0.526 - ETA: 454s - loss: 0.6324 - acc: 0.607 - ETA: 431s - loss: 0.6167 - acc: 0.632 - ETA: 418s - loss: 0.6089 - acc: 0.652 - ETA: 408s - loss: 0.6049 - acc: 0.661 - ETA: 400s - loss: 0.6007 - acc: 0.668 - ETA: 395s - loss: 0.6037 - acc: 0.668 - ETA: 389s - loss: 0.5874 - acc: 0.682 - ETA: 385s - loss: 0.5754 - acc: 0.693 - ETA: 381s - loss: 0.5747 - acc: 0.693 - ETA: 377s - loss: 0.5739 - acc: 0.699 - ETA: 375s - loss: 0.5739 - acc: 0.701 - ETA: 372s - loss: 0.5677 - acc: 0.708 - ETA: 369s - loss: 0.5688 - acc: 0.702 - ETA: 366s - loss: 0.5646 - acc: 0.708 - ETA: 364s - loss: 0.5610 - acc: 0.708 - ETA: 362s - loss: 0.5611 - acc: 0.709 - ETA: 360s - loss: 0.5568 - acc: 0.713 - ETA: 358s - loss: 0.5501 - acc: 0.720 - ETA: 356s - loss: 0.5451 - acc: 0.725 - ETA: 354s - loss: 0.5420 - acc: 0.728 - ETA: 352s - loss: 0.5450 - acc: 0.726 - ETA: 350s - loss: 0.5402 - acc: 0.7

6528/8643 [=====================>........] - ETA: 501s - loss: 0.3682 - acc: 0.833 - ETA: 464s - loss: 0.3237 - acc: 0.854 - ETA: 438s - loss: 0.2880 - acc: 0.881 - ETA: 420s - loss: 0.2808 - acc: 0.888 - ETA: 414s - loss: 0.2648 - acc: 0.891 - ETA: 406s - loss: 0.2381 - acc: 0.906 - ETA: 399s - loss: 0.2339 - acc: 0.909 - ETA: 394s - loss: 0.2269 - acc: 0.916 - ETA: 399s - loss: 0.2217 - acc: 0.917 - ETA: 396s - loss: 0.2201 - acc: 0.917 - ETA: 395s - loss: 0.2166 - acc: 0.921 - ETA: 395s - loss: 0.2201 - acc: 0.919 - ETA: 394s - loss: 0.2188 - acc: 0.919 - ETA: 392s - loss: 0.2220 - acc: 0.916 - ETA: 389s - loss: 0.2183 - acc: 0.918 - ETA: 392s - loss: 0.2116 - acc: 0.921 - ETA: 405s - loss: 0.2108 - acc: 0.919 - ETA: 413s - loss: 0.2066 - acc: 0.920 - ETA: 420s - loss: 0.2018 - acc: 0.922 - ETA: 426s - loss: 0.2053 - acc: 0.920 - ETA: 432s - loss: 0.2069 - acc: 0.920 - ETA: 436s - loss: 0.2028 - acc: 0.923 - ETA: 439s - loss: 0.2015 - acc: 0.924 - ETA: 441s - loss: 0.2032 - acc: 0.9

8643/8643 [==============================] - ETA: 142s - loss: 0.2151 - acc: 0.913 - ETA: 140s - loss: 0.2152 - acc: 0.914 - ETA: 138s - loss: 0.2156 - acc: 0.914 - ETA: 135s - loss: 0.2159 - acc: 0.913 - ETA: 133s - loss: 0.2153 - acc: 0.914 - ETA: 131s - loss: 0.2153 - acc: 0.914 - ETA: 129s - loss: 0.2155 - acc: 0.914 - ETA: 127s - loss: 0.2153 - acc: 0.914 - ETA: 124s - loss: 0.2152 - acc: 0.914 - ETA: 122s - loss: 0.2148 - acc: 0.914 - ETA: 120s - loss: 0.2154 - acc: 0.914 - ETA: 118s - loss: 0.2152 - acc: 0.914 - ETA: 116s - loss: 0.2153 - acc: 0.914 - ETA: 114s - loss: 0.2153 - acc: 0.913 - ETA: 111s - loss: 0.2151 - acc: 0.914 - ETA: 109s - loss: 0.2145 - acc: 0.914 - ETA: 107s - loss: 0.2146 - acc: 0.914 - ETA: 105s - loss: 0.2146 - acc: 0.914 - ETA: 103s - loss: 0.2148 - acc: 0.914 - ETA: 100s - loss: 0.2148 - acc: 0.913 - ETA: 98s - loss: 0.2147 - acc: 0.914 - ETA: 96s - loss: 0.2145 - acc: 0.91 - ETA: 94s - loss: 0.2145 - acc: 0.91 - ETA: 92s - loss: 0.2143 - acc: 0.91 - ET

6528/8643 [=====================>........] - ETA: 606s - loss: 0.0960 - acc: 0.989 - ETA: 597s - loss: 0.1119 - acc: 0.968 - ETA: 602s - loss: 0.1044 - acc: 0.972 - ETA: 600s - loss: 0.0956 - acc: 0.974 - ETA: 596s - loss: 0.1031 - acc: 0.968 - ETA: 592s - loss: 0.1167 - acc: 0.963 - ETA: 590s - loss: 0.1159 - acc: 0.959 - ETA: 587s - loss: 0.1159 - acc: 0.959 - ETA: 584s - loss: 0.1193 - acc: 0.961 - ETA: 582s - loss: 0.1136 - acc: 0.963 - ETA: 579s - loss: 0.1101 - acc: 0.965 - ETA: 576s - loss: 0.1105 - acc: 0.966 - ETA: 573s - loss: 0.1079 - acc: 0.967 - ETA: 571s - loss: 0.1080 - acc: 0.967 - ETA: 568s - loss: 0.1068 - acc: 0.966 - ETA: 565s - loss: 0.1047 - acc: 0.966 - ETA: 563s - loss: 0.1037 - acc: 0.966 - ETA: 561s - loss: 0.1019 - acc: 0.967 - ETA: 560s - loss: 0.1028 - acc: 0.966 - ETA: 558s - loss: 0.1018 - acc: 0.966 - ETA: 557s - loss: 0.0996 - acc: 0.967 - ETA: 554s - loss: 0.1020 - acc: 0.966 - ETA: 552s - loss: 0.1022 - acc: 0.966 - ETA: 549s - loss: 0.1014 - acc: 0.9

...

8643/8643 [==============================] - ETA: 124s - loss: 0.0078 - acc: 0.997 - ETA: 122s - loss: 0.0078 - acc: 0.997 - ETA: 120s - loss: 0.0077 - acc: 0.997 - ETA: 118s - loss: 0.0077 - acc: 0.997 - ETA: 116s - loss: 0.0077 - acc: 0.997 - ETA: 114s - loss: 0.0076 - acc: 0.997 - ETA: 112s - loss: 0.0076 - acc: 0.997 - ETA: 110s - loss: 0.0076 - acc: 0.997 - ETA: 108s - loss: 0.0075 - acc: 0.997 - ETA: 106s - loss: 0.0075 - acc: 0.997 - ETA: 104s - loss: 0.0076 - acc: 0.997 - ETA: 102s - loss: 0.0076 - acc: 0.997 - ETA: 100s - loss: 0.0075 - acc: 0.997 - ETA: 98s - loss: 0.0075 - acc: 0.997 - ETA: 96s - loss: 0.0075 - acc: 0.99 - ETA: 94s - loss: 0.0076 - acc: 0.99 - ETA: 92s - loss: 0.0076 - acc: 0.99 - ETA: 90s - loss: 0.0076 - acc: 0.99 - ETA: 88s - loss: 0.0077 - acc: 0.99 - ETA: 86s - loss: 0.0077 - acc: 0.99 - ETA: 85s - loss: 0.0077 - acc: 0.99 - ETA: 83s - loss: 0.0077 - acc: 0.99 - ETA: 81s - loss: 0.0079 - acc: 0.99 - ETA: 79s - loss: 0.0079 - acc: 0.99 - ETA: 77s - loss:

6528/8643 [=====================>........] - ETA: 413s - loss: 0.0143 - acc: 1.000 - ETA: 397s - loss: 0.0094 - acc: 1.000 - ETA: 387s - loss: 0.0065 - acc: 1.000 - ETA: 384s - loss: 0.0053 - acc: 1.000 - ETA: 389s - loss: 0.0043 - acc: 1.000 - ETA: 386s - loss: 0.0037 - acc: 1.000 - ETA: 383s - loss: 0.0033 - acc: 1.000 - ETA: 383s - loss: 0.0030 - acc: 1.000 - ETA: 381s - loss: 0.0027 - acc: 1.000 - ETA: 382s - loss: 0.0045 - acc: 0.997 - ETA: 381s - loss: 0.0043 - acc: 0.998 - ETA: 384s - loss: 0.0040 - acc: 0.998 - ETA: 387s - loss: 0.0039 - acc: 0.998 - ETA: 384s - loss: 0.0069 - acc: 0.997 - ETA: 386s - loss: 0.0065 - acc: 0.997 - ETA: 388s - loss: 0.0061 - acc: 0.997 - ETA: 385s - loss: 0.0058 - acc: 0.997 - ETA: 381s - loss: 0.0055 - acc: 0.997 - ETA: 378s - loss: 0.0053 - acc: 0.997 - ETA: 376s - loss: 0.0053 - acc: 0.997 - ETA: 373s - loss: 0.0063 - acc: 0.997 - ETA: 370s - loss: 0.0060 - acc: 0.997 - ETA: 367s - loss: 0.0064 - acc: 0.997 - ETA: 364s - loss: 0.0062 - acc: 0.9

6528/8643 [=====================>........] - ETA: 363s - loss: 2.7577e-04 - acc: 1.000 - ETA: 362s - loss: 0.0012 - acc: 1.0000    - ETA: 365s - loss: 0.0040 - acc: 1.000 - ETA: 362s - loss: 0.0050 - acc: 1.000 - ETA: 360s - loss: 0.0071 - acc: 1.000 - ETA: 358s - loss: 0.0073 - acc: 1.000 - ETA: 357s - loss: 0.0064 - acc: 1.000 - ETA: 355s - loss: 0.0057 - acc: 1.000 - ETA: 354s - loss: 0.0053 - acc: 1.000 - ETA: 352s - loss: 0.0048 - acc: 1.000 - ETA: 351s - loss: 0.0046 - acc: 1.000 - ETA: 350s - loss: 0.0043 - acc: 1.000 - ETA: 348s - loss: 0.0042 - acc: 1.000 - ETA: 346s - loss: 0.0050 - acc: 0.999 - ETA: 346s - loss: 0.0047 - acc: 0.999 - ETA: 344s - loss: 0.0045 - acc: 0.999 - ETA: 343s - loss: 0.0058 - acc: 0.999 - ETA: 342s - loss: 0.0062 - acc: 0.998 - ETA: 340s - loss: 0.0059 - acc: 0.998 - ETA: 339s - loss: 0.0056 - acc: 0.999 - ETA: 338s - loss: 0.0054 - acc: 0.999 - ETA: 337s - loss: 0.0055 - acc: 0.999 - ETA: 335s - loss: 0.0054 - acc: 0.999 - ETA: 334s - loss: 0.0054 - 

8643/8643 [==============================] - ETA: 89s - loss: 0.0072 - acc: 0.99 - ETA: 88s - loss: 0.0076 - acc: 0.99 - ETA: 86s - loss: 0.0076 - acc: 0.99 - ETA: 85s - loss: 0.0075 - acc: 0.99 - ETA: 84s - loss: 0.0075 - acc: 0.99 - ETA: 82s - loss: 0.0075 - acc: 0.99 - ETA: 81s - loss: 0.0075 - acc: 0.99 - ETA: 79s - loss: 0.0074 - acc: 0.99 - ETA: 78s - loss: 0.0074 - acc: 0.99 - ETA: 77s - loss: 0.0078 - acc: 0.99 - ETA: 75s - loss: 0.0078 - acc: 0.99 - ETA: 74s - loss: 0.0078 - acc: 0.99 - ETA: 72s - loss: 0.0078 - acc: 0.99 - ETA: 71s - loss: 0.0077 - acc: 0.99 - ETA: 70s - loss: 0.0082 - acc: 0.99 - ETA: 68s - loss: 0.0081 - acc: 0.99 - ETA: 67s - loss: 0.0081 - acc: 0.99 - ETA: 66s - loss: 0.0081 - acc: 0.99 - ETA: 64s - loss: 0.0080 - acc: 0.99 - ETA: 63s - loss: 0.0081 - acc: 0.99 - ETA: 61s - loss: 0.0081 - acc: 0.99 - ETA: 60s - loss: 0.0081 - acc: 0.99 - ETA: 59s - loss: 0.0080 - acc: 0.99 - ETA: 57s - loss: 0.0080 - acc: 0.99 - ETA: 56s - loss: 0.0080 - acc: 0.99 - ETA: 

6528/8643 [=====================>........] - ETA: 361s - loss: 3.5911e-04 - acc: 1.000 - ETA: 369s - loss: 0.0075 - acc: 1.0000    - ETA: 368s - loss: 0.0072 - acc: 1.000 - ETA: 367s - loss: 0.0059 - acc: 1.000 - ETA: 366s - loss: 0.0048 - acc: 1.000 - ETA: 364s - loss: 0.0044 - acc: 1.000 - ETA: 362s - loss: 0.0039 - acc: 1.000 - ETA: 359s - loss: 0.0035 - acc: 1.000 - ETA: 358s - loss: 0.0045 - acc: 0.998 - ETA: 357s - loss: 0.0041 - acc: 0.999 - ETA: 356s - loss: 0.0039 - acc: 0.999 - ETA: 354s - loss: 0.0036 - acc: 0.999 - ETA: 353s - loss: 0.0044 - acc: 0.999 - ETA: 352s - loss: 0.0041 - acc: 0.999 - ETA: 351s - loss: 0.0039 - acc: 0.999 - ETA: 349s - loss: 0.0037 - acc: 0.999 - ETA: 348s - loss: 0.0036 - acc: 0.999 - ETA: 347s - loss: 0.0042 - acc: 0.998 - ETA: 345s - loss: 0.0041 - acc: 0.998 - ETA: 344s - loss: 0.0040 - acc: 0.999 - ETA: 343s - loss: 0.0038 - acc: 0.999 - ETA: 342s - loss: 0.0037 - acc: 0.999 - ETA: 341s - loss: 0.0039 - acc: 0.999 - ETA: 339s - loss: 0.0037 - 

In [17]:
y_pred = model.predict_classes(X_test) - 1

2247/2247 [==============================] - ETA: 64 - ETA: 41 - ETA: 34 - ETA: 29 - ETA: 27 - ETA: 25 - ETA: 24 - ETA: 23 - ETA: 22 - ETA: 21 - ETA: 20 - ETA: 19 - ETA: 19 - ETA: 18 - ETA: 18 - ETA: 17 - ETA: 17 - ETA: 16 - ETA: 16 - ETA: 16 - ETA: 15 - ETA: 15 - ETA: 14 - ETA: 14 - ETA: 14 - ETA: 13 - ETA: 13 - ETA: 13 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 9 - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 20s    


In [18]:
accuracy_score(y_pred, test_labels)

0.6497552291944815

## Convolutional model with pretrained fasttext vectors
for ttk

prepare weights matrix for embedding layer

In [16]:
from gensim.models import FastText

emb_model = FastText.load('data/fasttext/araneum_none_fasttextskipgram_300_5_2018.model')

In [17]:
word_vectors = emb_model.wv

In [51]:
EMBEDDING_DIM = 300
NUM_WORDS = 20000
vocabulary_size = len(word_index)+1
embedding_matrix = np.zeros((vocabulary_size, EMBEDDING_DIM))
for word, i in word_index.items():
    if i>=NUM_WORDS:
        continue
    try:
        embedding_vector = word_vectors[word]
        embedding_matrix[i] = embedding_vector
    except KeyError:
        embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),EMBEDDING_DIM)

prepare and train model

In [52]:
model = Sequential()

model.add(Embedding(vocabulary_size, EMBEDDING_DIM, weights=[embedding_matrix], trainable=True, input_length=100))
model.add(Dropout(0.2))
model.add(Conv1D(256, 5, activation='relu'))
model.add(MaxPooling1D(pool_size=4))
model.add(LSTM(100))
model.add(Dense(3, activation='sigmoid'))


model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=32, epochs=5)

Epoch 1/5


6592/8643 [=====================>........] - ETA: 2006s - loss: 0.6920 - acc: 0.62 - ETA: 1130s - loss: 0.6533 - acc: 0.68 - ETA: 838s - loss: 0.6062 - acc: 0.7222 - ETA: 690s - loss: 0.6035 - acc: 0.713 - ETA: 596s - loss: 0.6147 - acc: 0.700 - ETA: 532s - loss: 0.6264 - acc: 0.687 - ETA: 491s - loss: 0.6198 - acc: 0.683 - ETA: 460s - loss: 0.6182 - acc: 0.681 - ETA: 433s - loss: 0.6099 - acc: 0.679 - ETA: 412s - loss: 0.6042 - acc: 0.681 - ETA: 393s - loss: 0.5995 - acc: 0.690 - ETA: 378s - loss: 0.5914 - acc: 0.702 - ETA: 365s - loss: 0.5796 - acc: 0.712 - ETA: 353s - loss: 0.5791 - acc: 0.713 - ETA: 344s - loss: 0.5738 - acc: 0.718 - ETA: 334s - loss: 0.5722 - acc: 0.719 - ETA: 328s - loss: 0.5673 - acc: 0.722 - ETA: 320s - loss: 0.5686 - acc: 0.721 - ETA: 314s - loss: 0.5654 - acc: 0.725 - ETA: 308s - loss: 0.5660 - acc: 0.725 - ETA: 302s - loss: 0.5640 - acc: 0.726 - ETA: 297s - loss: 0.5607 - acc: 0.729 - ETA: 292s - loss: 0.5590 - acc: 0.729 - ETA: 288s - loss: 0.5540 - acc: 0.

6592/8643 [=====================>........] - ETA: 248s - loss: 0.1943 - acc: 0.916 - ETA: 246s - loss: 0.2888 - acc: 0.875 - ETA: 244s - loss: 0.2894 - acc: 0.878 - ETA: 244s - loss: 0.2906 - acc: 0.880 - ETA: 244s - loss: 0.2894 - acc: 0.877 - ETA: 245s - loss: 0.2725 - acc: 0.890 - ETA: 245s - loss: 0.2679 - acc: 0.892 - ETA: 244s - loss: 0.2770 - acc: 0.890 - ETA: 244s - loss: 0.2896 - acc: 0.883 - ETA: 242s - loss: 0.3199 - acc: 0.870 - ETA: 242s - loss: 0.3235 - acc: 0.864 - ETA: 241s - loss: 0.3124 - acc: 0.870 - ETA: 243s - loss: 0.3040 - acc: 0.874 - ETA: 242s - loss: 0.3033 - acc: 0.877 - ETA: 243s - loss: 0.3116 - acc: 0.872 - ETA: 241s - loss: 0.3136 - acc: 0.870 - ETA: 243s - loss: 0.3079 - acc: 0.873 - ETA: 243s - loss: 0.3070 - acc: 0.875 - ETA: 243s - loss: 0.3007 - acc: 0.877 - ETA: 244s - loss: 0.2989 - acc: 0.879 - ETA: 243s - loss: 0.3026 - acc: 0.877 - ETA: 241s - loss: 0.3043 - acc: 0.877 - ETA: 241s - loss: 0.3002 - acc: 0.879 - ETA: 241s - loss: 0.2979 - acc: 0.8

8643/8643 [==============================] - ETA: 55s - loss: 0.2770 - acc: 0.88 - ETA: 54s - loss: 0.2773 - acc: 0.88 - ETA: 53s - loss: 0.2771 - acc: 0.88 - ETA: 52s - loss: 0.2769 - acc: 0.88 - ETA: 51s - loss: 0.2765 - acc: 0.88 - ETA: 50s - loss: 0.2771 - acc: 0.88 - ETA: 49s - loss: 0.2769 - acc: 0.88 - ETA: 48s - loss: 0.2769 - acc: 0.88 - ETA: 47s - loss: 0.2767 - acc: 0.88 - ETA: 46s - loss: 0.2764 - acc: 0.88 - ETA: 46s - loss: 0.2763 - acc: 0.88 - ETA: 45s - loss: 0.2765 - acc: 0.88 - ETA: 44s - loss: 0.2763 - acc: 0.88 - ETA: 43s - loss: 0.2759 - acc: 0.88 - ETA: 42s - loss: 0.2763 - acc: 0.88 - ETA: 41s - loss: 0.2761 - acc: 0.88 - ETA: 40s - loss: 0.2765 - acc: 0.88 - ETA: 39s - loss: 0.2761 - acc: 0.88 - ETA: 38s - loss: 0.2758 - acc: 0.88 - ETA: 37s - loss: 0.2760 - acc: 0.88 - ETA: 36s - loss: 0.2767 - acc: 0.88 - ETA: 35s - loss: 0.2770 - acc: 0.88 - ETA: 35s - loss: 0.2767 - acc: 0.88 - ETA: 34s - loss: 0.2770 - acc: 0.88 - ETA: 33s - loss: 0.2771 - acc: 0.88 - ETA: 

6752/8643 [======================>.......] - ETA: 153s - loss: 0.1315 - acc: 0.968 - ETA: 153s - loss: 0.1028 - acc: 0.968 - ETA: 151s - loss: 0.1409 - acc: 0.961 - ETA: 155s - loss: 0.1452 - acc: 0.955 - ETA: 154s - loss: 0.1795 - acc: 0.947 - ETA: 151s - loss: 0.1757 - acc: 0.946 - ETA: 154s - loss: 0.1906 - acc: 0.930 - ETA: 152s - loss: 0.2067 - acc: 0.925 - ETA: 150s - loss: 0.1975 - acc: 0.925 - ETA: 148s - loss: 0.1869 - acc: 0.930 - ETA: 146s - loss: 0.1845 - acc: 0.930 - ETA: 145s - loss: 0.1766 - acc: 0.933 - ETA: 144s - loss: 0.1715 - acc: 0.934 - ETA: 143s - loss: 0.1677 - acc: 0.936 - ETA: 142s - loss: 0.1663 - acc: 0.935 - ETA: 141s - loss: 0.1618 - acc: 0.938 - ETA: 140s - loss: 0.1616 - acc: 0.938 - ETA: 139s - loss: 0.1571 - acc: 0.941 - ETA: 138s - loss: 0.1622 - acc: 0.941 - ETA: 137s - loss: 0.1614 - acc: 0.941 - ETA: 137s - loss: 0.1589 - acc: 0.942 - ETA: 136s - loss: 0.1639 - acc: 0.940 - ETA: 135s - loss: 0.1583 - acc: 0.942 - ETA: 134s - loss: 0.1567 - acc: 0.9

6720/8643 [======================>.......] - ETA: 169s - loss: 0.0736 - acc: 1.000 - ETA: 163s - loss: 0.0916 - acc: 0.974 - ETA: 156s - loss: 0.0841 - acc: 0.979 - ETA: 155s - loss: 0.1030 - acc: 0.971 - ETA: 153s - loss: 0.0946 - acc: 0.975 - ETA: 152s - loss: 0.0954 - acc: 0.972 - ETA: 151s - loss: 0.0863 - acc: 0.976 - ETA: 149s - loss: 0.0788 - acc: 0.979 - ETA: 149s - loss: 0.0787 - acc: 0.979 - ETA: 148s - loss: 0.0785 - acc: 0.980 - ETA: 147s - loss: 0.0798 - acc: 0.980 - ETA: 148s - loss: 0.0855 - acc: 0.976 - ETA: 148s - loss: 0.0951 - acc: 0.973 - ETA: 147s - loss: 0.1017 - acc: 0.970 - ETA: 148s - loss: 0.1048 - acc: 0.969 - ETA: 149s - loss: 0.1040 - acc: 0.969 - ETA: 150s - loss: 0.1066 - acc: 0.966 - ETA: 149s - loss: 0.1022 - acc: 0.968 - ETA: 148s - loss: 0.0999 - acc: 0.968 - ETA: 147s - loss: 0.0994 - acc: 0.967 - ETA: 147s - loss: 0.1001 - acc: 0.967 - ETA: 146s - loss: 0.0979 - acc: 0.968 - ETA: 144s - loss: 0.0976 - acc: 0.968 - ETA: 144s - loss: 0.1019 - acc: 0.9

8643/8643 [==============================] - ETA: 32s - loss: 0.1107 - acc: 0.95 - ETA: 31s - loss: 0.1104 - acc: 0.95 - ETA: 31s - loss: 0.1104 - acc: 0.95 - ETA: 30s - loss: 0.1105 - acc: 0.95 - ETA: 30s - loss: 0.1106 - acc: 0.95 - ETA: 29s - loss: 0.1108 - acc: 0.95 - ETA: 29s - loss: 0.1112 - acc: 0.95 - ETA: 28s - loss: 0.1115 - acc: 0.95 - ETA: 28s - loss: 0.1117 - acc: 0.95 - ETA: 27s - loss: 0.1121 - acc: 0.95 - ETA: 26s - loss: 0.1124 - acc: 0.95 - ETA: 26s - loss: 0.1120 - acc: 0.95 - ETA: 25s - loss: 0.1117 - acc: 0.95 - ETA: 25s - loss: 0.1116 - acc: 0.95 - ETA: 24s - loss: 0.1115 - acc: 0.95 - ETA: 24s - loss: 0.1116 - acc: 0.95 - ETA: 23s - loss: 0.1117 - acc: 0.95 - ETA: 23s - loss: 0.1116 - acc: 0.95 - ETA: 22s - loss: 0.1118 - acc: 0.95 - ETA: 21s - loss: 0.1119 - acc: 0.95 - ETA: 21s - loss: 0.1117 - acc: 0.95 - ETA: 20s - loss: 0.1121 - acc: 0.95 - ETA: 20s - loss: 0.1119 - acc: 0.95 - ETA: 19s - loss: 0.1119 - acc: 0.95 - ETA: 19s - loss: 0.1118 - acc: 0.95 - ETA: 

6720/8643 [======================>.......] - ETA: 145s - loss: 0.0416 - acc: 0.989 - ETA: 159s - loss: 0.0573 - acc: 0.984 - ETA: 170s - loss: 0.0896 - acc: 0.975 - ETA: 169s - loss: 0.0758 - acc: 0.981 - ETA: 166s - loss: 0.0704 - acc: 0.981 - ETA: 162s - loss: 0.0618 - acc: 0.984 - ETA: 158s - loss: 0.0644 - acc: 0.983 - ETA: 155s - loss: 0.0585 - acc: 0.985 - ETA: 153s - loss: 0.0603 - acc: 0.983 - ETA: 151s - loss: 0.0612 - acc: 0.983 - ETA: 149s - loss: 0.0577 - acc: 0.984 - ETA: 147s - loss: 0.0643 - acc: 0.982 - ETA: 146s - loss: 0.0639 - acc: 0.981 - ETA: 145s - loss: 0.0603 - acc: 0.982 - ETA: 144s - loss: 0.0603 - acc: 0.983 - ETA: 143s - loss: 0.0628 - acc: 0.981 - ETA: 142s - loss: 0.0608 - acc: 0.982 - ETA: 141s - loss: 0.0595 - acc: 0.982 - ETA: 140s - loss: 0.0597 - acc: 0.981 - ETA: 139s - loss: 0.0576 - acc: 0.982 - ETA: 138s - loss: 0.0583 - acc: 0.982 - ETA: 137s - loss: 0.0575 - acc: 0.983 - ETA: 136s - loss: 0.0576 - acc: 0.983 - ETA: 136s - loss: 0.0574 - acc: 0.9

In [53]:
y_pred = model.predict_classes(X_test) - 1
accuracy_score(y_pred, test_labels)

2247/2247 [==============================] - ETA: 26 - ETA: 16 - ETA: 13 - ETA: 12 - ETA: 11 - ETA: 10 - ETA: 9 - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 8s     


0.6853582554517134

## Networks for bank

In [18]:
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(train_bank_ready)

x_bank_train = tokenizer.texts_to_sequences(train_bank_ready)
x_bank_test = tokenizer.texts_to_sequences(test_bank_ready)

X_bank_train = pad_sequences(x_bank_train, maxlen=100)
X_bank_test = pad_sequences(x_bank_test, maxlen=100)

word_bank_index = tokenizer.word_index

encoder = LabelBinarizer()
encoder.fit(train_bank_labels)
y_bank_train = encoder.transform(train_bank_labels)
y_bank_test = encoder.transform(test_bank_labels)

model without pretrained vectors

In [56]:
model = Sequential()

model.add(Embedding(10000, 1000, input_length=100))
model.add(Dropout(0.2))
model.add(Conv1D(256, 5, activation='relu'))
model.add(MaxPooling1D(pool_size=4))
model.add(LSTM(100))
model.add(Dense(3, activation='sigmoid'))


model.compile(loss='binary_crossentropy', optimizer='adam',    metrics=['accuracy'])

model.fit(X_bank_train, y_bank_train, batch_size=32, epochs=5)

Epoch 1/5


6528/9392 [===================>..........] - ETA: 1003s - loss: 0.6933 - acc: 0.45 - ETA: 716s - loss: 0.6436 - acc: 0.6563 - ETA: 619s - loss: 0.5923 - acc: 0.722 - ETA: 568s - loss: 0.5621 - acc: 0.744 - ETA: 536s - loss: 0.5536 - acc: 0.750 - ETA: 514s - loss: 0.5260 - acc: 0.767 - ETA: 507s - loss: 0.5112 - acc: 0.776 - ETA: 501s - loss: 0.5222 - acc: 0.773 - ETA: 496s - loss: 0.4957 - acc: 0.787 - ETA: 487s - loss: 0.4727 - acc: 0.800 - ETA: 480s - loss: 0.4719 - acc: 0.799 - ETA: 474s - loss: 0.4605 - acc: 0.805 - ETA: 467s - loss: 0.4512 - acc: 0.810 - ETA: 460s - loss: 0.4508 - acc: 0.812 - ETA: 455s - loss: 0.4442 - acc: 0.815 - ETA: 449s - loss: 0.4419 - acc: 0.816 - ETA: 444s - loss: 0.4398 - acc: 0.816 - ETA: 440s - loss: 0.4326 - acc: 0.820 - ETA: 435s - loss: 0.4398 - acc: 0.815 - ETA: 431s - loss: 0.4314 - acc: 0.819 - ETA: 427s - loss: 0.4270 - acc: 0.822 - ETA: 423s - loss: 0.4234 - acc: 0.824 - ETA: 419s - loss: 0.4212 - acc: 0.825 - ETA: 416s - loss: 0.4165 - acc: 0.

6528/9392 [===================>..........] - ETA: 404s - loss: 0.1522 - acc: 0.958 - ETA: 401s - loss: 0.1154 - acc: 0.974 - ETA: 402s - loss: 0.1003 - acc: 0.975 - ETA: 400s - loss: 0.0866 - acc: 0.981 - ETA: 398s - loss: 0.0856 - acc: 0.981 - ETA: 398s - loss: 0.1250 - acc: 0.972 - ETA: 397s - loss: 0.1371 - acc: 0.962 - ETA: 395s - loss: 0.1324 - acc: 0.963 - ETA: 394s - loss: 0.1319 - acc: 0.964 - ETA: 392s - loss: 0.1243 - acc: 0.965 - ETA: 391s - loss: 0.1184 - acc: 0.966 - ETA: 391s - loss: 0.1178 - acc: 0.967 - ETA: 389s - loss: 0.1121 - acc: 0.968 - ETA: 388s - loss: 0.1125 - acc: 0.968 - ETA: 387s - loss: 0.1120 - acc: 0.967 - ETA: 385s - loss: 0.1185 - acc: 0.964 - ETA: 383s - loss: 0.1177 - acc: 0.963 - ETA: 382s - loss: 0.1182 - acc: 0.961 - ETA: 381s - loss: 0.1241 - acc: 0.958 - ETA: 379s - loss: 0.1240 - acc: 0.958 - ETA: 378s - loss: 0.1291 - acc: 0.958 - ETA: 376s - loss: 0.1322 - acc: 0.957 - ETA: 375s - loss: 0.1351 - acc: 0.956 - ETA: 374s - loss: 0.1330 - acc: 0.9

9392/9392 [==============================] - ETA: 123s - loss: 0.1266 - acc: 0.953 - ETA: 122s - loss: 0.1270 - acc: 0.953 - ETA: 121s - loss: 0.1268 - acc: 0.953 - ETA: 119s - loss: 0.1266 - acc: 0.953 - ETA: 118s - loss: 0.1263 - acc: 0.953 - ETA: 116s - loss: 0.1261 - acc: 0.953 - ETA: 115s - loss: 0.1260 - acc: 0.953 - ETA: 114s - loss: 0.1255 - acc: 0.953 - ETA: 112s - loss: 0.1258 - acc: 0.953 - ETA: 111s - loss: 0.1257 - acc: 0.953 - ETA: 109s - loss: 0.1259 - acc: 0.953 - ETA: 108s - loss: 0.1261 - acc: 0.952 - ETA: 107s - loss: 0.1259 - acc: 0.953 - ETA: 105s - loss: 0.1258 - acc: 0.953 - ETA: 104s - loss: 0.1258 - acc: 0.953 - ETA: 102s - loss: 0.1256 - acc: 0.953 - ETA: 101s - loss: 0.1252 - acc: 0.953 - ETA: 100s - loss: 0.1249 - acc: 0.953 - ETA: 98s - loss: 0.1253 - acc: 0.953 - ETA: 97s - loss: 0.1257 - acc: 0.95 - ETA: 95s - loss: 0.1254 - acc: 0.95 - ETA: 94s - loss: 0.1253 - acc: 0.95 - ETA: 92s - loss: 0.1252 - acc: 0.95 - ETA: 91s - loss: 0.1254 - acc: 0.95 - ETA: 9

6528/9392 [===================>..........] - ETA: 400s - loss: 0.0299 - acc: 0.989 - ETA: 402s - loss: 0.0314 - acc: 0.989 - ETA: 402s - loss: 0.0314 - acc: 0.989 - ETA: 401s - loss: 0.0578 - acc: 0.974 - ETA: 405s - loss: 0.0583 - acc: 0.975 - ETA: 404s - loss: 0.0562 - acc: 0.979 - ETA: 401s - loss: 0.0521 - acc: 0.982 - ETA: 400s - loss: 0.0482 - acc: 0.984 - ETA: 398s - loss: 0.0517 - acc: 0.982 - ETA: 396s - loss: 0.0571 - acc: 0.981 - ETA: 395s - loss: 0.0567 - acc: 0.980 - ETA: 393s - loss: 0.0665 - acc: 0.977 - ETA: 392s - loss: 0.0674 - acc: 0.976 - ETA: 391s - loss: 0.0636 - acc: 0.977 - ETA: 389s - loss: 0.0619 - acc: 0.977 - ETA: 387s - loss: 0.0582 - acc: 0.979 - ETA: 386s - loss: 0.0567 - acc: 0.980 - ETA: 384s - loss: 0.0561 - acc: 0.980 - ETA: 383s - loss: 0.0538 - acc: 0.981 - ETA: 381s - loss: 0.0586 - acc: 0.980 - ETA: 380s - loss: 0.0567 - acc: 0.981 - ETA: 378s - loss: 0.0555 - acc: 0.981 - ETA: 377s - loss: 0.0562 - acc: 0.981 - ETA: 376s - loss: 0.0584 - acc: 0.9

6528/9392 [===================>..........] - ETA: 402s - loss: 0.0303 - acc: 1.000 - ETA: 401s - loss: 0.0635 - acc: 0.989 - ETA: 402s - loss: 0.0468 - acc: 0.993 - ETA: 406s - loss: 0.0355 - acc: 0.994 - ETA: 406s - loss: 0.0306 - acc: 0.995 - ETA: 404s - loss: 0.0265 - acc: 0.996 - ETA: 401s - loss: 0.0235 - acc: 0.997 - ETA: 400s - loss: 0.0219 - acc: 0.997 - ETA: 398s - loss: 0.0227 - acc: 0.995 - ETA: 396s - loss: 0.0209 - acc: 0.995 - ETA: 394s - loss: 0.0195 - acc: 0.996 - ETA: 393s - loss: 0.0215 - acc: 0.994 - ETA: 391s - loss: 0.0231 - acc: 0.993 - ETA: 391s - loss: 0.0239 - acc: 0.993 - ETA: 390s - loss: 0.0242 - acc: 0.993 - ETA: 389s - loss: 0.0236 - acc: 0.993 - ETA: 387s - loss: 0.0224 - acc: 0.993 - ETA: 386s - loss: 0.0250 - acc: 0.993 - ETA: 384s - loss: 0.0239 - acc: 0.993 - ETA: 383s - loss: 0.0229 - acc: 0.993 - ETA: 381s - loss: 0.0222 - acc: 0.994 - ETA: 380s - loss: 0.0217 - acc: 0.994 - ETA: 378s - loss: 0.0218 - acc: 0.993 - ETA: 377s - loss: 0.0211 - acc: 0.9

9392/9392 [==============================] - ETA: 123s - loss: 0.0208 - acc: 0.993 - ETA: 121s - loss: 0.0211 - acc: 0.993 - ETA: 120s - loss: 0.0210 - acc: 0.993 - ETA: 119s - loss: 0.0210 - acc: 0.993 - ETA: 117s - loss: 0.0214 - acc: 0.993 - ETA: 116s - loss: 0.0213 - acc: 0.993 - ETA: 114s - loss: 0.0213 - acc: 0.993 - ETA: 113s - loss: 0.0212 - acc: 0.993 - ETA: 112s - loss: 0.0211 - acc: 0.993 - ETA: 110s - loss: 0.0210 - acc: 0.993 - ETA: 109s - loss: 0.0210 - acc: 0.993 - ETA: 108s - loss: 0.0209 - acc: 0.993 - ETA: 106s - loss: 0.0209 - acc: 0.993 - ETA: 105s - loss: 0.0208 - acc: 0.993 - ETA: 103s - loss: 0.0210 - acc: 0.993 - ETA: 102s - loss: 0.0210 - acc: 0.993 - ETA: 101s - loss: 0.0210 - acc: 0.993 - ETA: 99s - loss: 0.0210 - acc: 0.993 - ETA: 98s - loss: 0.0211 - acc: 0.99 - ETA: 97s - loss: 0.0210 - acc: 0.99 - ETA: 95s - loss: 0.0209 - acc: 0.99 - ETA: 94s - loss: 0.0209 - acc: 0.99 - ETA: 92s - loss: 0.0208 - acc: 0.99 - ETA: 91s - loss: 0.0207 - acc: 0.99 - ETA: 90s

6528/9392 [===================>..........] - ETA: 403s - loss: 0.0051 - acc: 1.000 - ETA: 400s - loss: 0.0034 - acc: 1.000 - ETA: 404s - loss: 0.0327 - acc: 0.993 - ETA: 408s - loss: 0.0256 - acc: 0.994 - ETA: 405s - loss: 0.0210 - acc: 0.995 - ETA: 403s - loss: 0.0182 - acc: 0.996 - ETA: 401s - loss: 0.0161 - acc: 0.997 - ETA: 398s - loss: 0.0144 - acc: 0.997 - ETA: 397s - loss: 0.0131 - acc: 0.997 - ETA: 395s - loss: 0.0119 - acc: 0.997 - ETA: 394s - loss: 0.0110 - acc: 0.998 - ETA: 395s - loss: 0.0110 - acc: 0.998 - ETA: 393s - loss: 0.0103 - acc: 0.998 - ETA: 391s - loss: 0.0097 - acc: 0.998 - ETA: 389s - loss: 0.0094 - acc: 0.998 - ETA: 388s - loss: 0.0094 - acc: 0.998 - ETA: 386s - loss: 0.0092 - acc: 0.998 - ETA: 385s - loss: 0.0142 - acc: 0.997 - ETA: 384s - loss: 0.0139 - acc: 0.997 - ETA: 382s - loss: 0.0133 - acc: 0.997 - ETA: 381s - loss: 0.0132 - acc: 0.998 - ETA: 379s - loss: 0.0136 - acc: 0.997 - ETA: 378s - loss: 0.0131 - acc: 0.997 - ETA: 376s - loss: 0.0127 - acc: 0.9

In [57]:
y_bank_pred = model.predict_classes(X_bank_test) - 1
accuracy_score(y_bank_pred, test_bank_labels)

3313/3313 [==============================] - ETA: 82 - ETA: 55 - ETA: 47 - ETA: 42 - ETA: 39 - ETA: 37 - ETA: 35 - ETA: 34 - ETA: 33 - ETA: 32 - ETA: 31 - ETA: 31 - ETA: 30 - ETA: 29 - ETA: 29 - ETA: 28 - ETA: 28 - ETA: 27 - ETA: 27 - ETA: 26 - ETA: 26 - ETA: 26 - ETA: 25 - ETA: 25 - ETA: 25 - ETA: 24 - ETA: 24 - ETA: 23 - ETA: 23 - ETA: 23 - ETA: 22 - ETA: 22 - ETA: 22 - ETA: 21 - ETA: 21 - ETA: 21 - ETA: 20 - ETA: 20 - ETA: 20 - ETA: 19 - ETA: 19 - ETA: 19 - ETA: 18 - ETA: 18 - ETA: 18 - ETA: 17 - ETA: 17 - ETA: 17 - ETA: 16 - ETA: 16 - ETA: 16 - ETA: 15 - ETA: 15 - ETA: 15 - ETA: 14 - ETA: 14 - ETA: 14 - ETA: 13 - ETA: 13 - ETA: 13 - ETA: 13 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 9 - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  

0.6978569272562632

with pretrained

In [19]:
EMBEDDING_DIM = 300
NUM_WORDS = 20000
vocabulary_size = len(word_bank_index)+1
embedding_matrix = np.zeros((vocabulary_size, EMBEDDING_DIM))
for word, i in word_bank_index.items():
    if i>=NUM_WORDS:
        continue
    try:
        embedding_vector = word_vectors[word]
        embedding_matrix[i] = embedding_vector
    except KeyError:
        embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),EMBEDDING_DIM)

In [20]:
model = Sequential()

model.add(Embedding(vocabulary_size, EMBEDDING_DIM, weights=[embedding_matrix], trainable=True, input_length=100))
model.add(Dropout(0.2))
model.add(Conv1D(256, 5, activation='relu'))
model.add(MaxPooling1D(pool_size=4))
model.add(LSTM(100))
model.add(Dense(3, activation='sigmoid'))


model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_bank_train, y_bank_train, batch_size=64, epochs=5)

Epoch 1/5
9392/9392 [==============================] - ETA: 623s - loss: 0.6956 - acc: 0.218 - ETA: 448s - loss: 0.6516 - acc: 0.489 - ETA: 392s - loss: 0.6113 - acc: 0.600 - ETA: 359s - loss: 0.5719 - acc: 0.656 - ETA: 333s - loss: 0.5573 - acc: 0.681 - ETA: 318s - loss: 0.5192 - acc: 0.720 - ETA: 305s - loss: 0.5078 - acc: 0.735 - ETA: 295s - loss: 0.5032 - acc: 0.746 - ETA: 286s - loss: 0.4929 - acc: 0.755 - ETA: 278s - loss: 0.4964 - acc: 0.754 - ETA: 272s - loss: 0.4866 - acc: 0.762 - ETA: 268s - loss: 0.4794 - acc: 0.771 - ETA: 262s - loss: 0.4771 - acc: 0.772 - ETA: 258s - loss: 0.4732 - acc: 0.776 - ETA: 254s - loss: 0.4669 - acc: 0.780 - ETA: 250s - loss: 0.4593 - acc: 0.785 - ETA: 248s - loss: 0.4549 - acc: 0.788 - ETA: 245s - loss: 0.4547 - acc: 0.788 - ETA: 241s - loss: 0.4486 - acc: 0.793 - ETA: 238s - loss: 0.4496 - acc: 0.793 - ETA: 235s - loss: 0.4468 - acc: 0.794 - ETA: 233s - loss: 0.4419 - acc: 0.797 - ETA: 230s - loss: 0.4393 - acc: 0.799 - ETA: 227s - loss: 0.4325 

9392/9392 [==============================] - ETA: 236s - loss: 0.1512 - acc: 0.932 - ETA: 233s - loss: 0.1551 - acc: 0.940 - ETA: 232s - loss: 0.1563 - acc: 0.939 - ETA: 231s - loss: 0.1721 - acc: 0.932 - ETA: 231s - loss: 0.1597 - acc: 0.940 - ETA: 230s - loss: 0.1607 - acc: 0.937 - ETA: 227s - loss: 0.1738 - acc: 0.927 - ETA: 225s - loss: 0.1868 - acc: 0.919 - ETA: 224s - loss: 0.1890 - acc: 0.917 - ETA: 223s - loss: 0.1846 - acc: 0.922 - ETA: 222s - loss: 0.1768 - acc: 0.926 - ETA: 220s - loss: 0.1783 - acc: 0.924 - ETA: 218s - loss: 0.1755 - acc: 0.925 - ETA: 217s - loss: 0.1760 - acc: 0.926 - ETA: 215s - loss: 0.1755 - acc: 0.926 - ETA: 212s - loss: 0.1763 - acc: 0.925 - ETA: 209s - loss: 0.1781 - acc: 0.925 - ETA: 206s - loss: 0.1798 - acc: 0.924 - ETA: 203s - loss: 0.1812 - acc: 0.923 - ETA: 200s - loss: 0.1798 - acc: 0.925 - ETA: 198s - loss: 0.1803 - acc: 0.924 - ETA: 195s - loss: 0.1823 - acc: 0.923 - ETA: 193s - loss: 0.1818 - acc: 0.924 - ETA: 190s - loss: 0.1828 - acc: 0.9

9392/9392 [==============================] - ETA: 233s - loss: 0.1672 - acc: 0.932 - ETA: 233s - loss: 0.1260 - acc: 0.960 - ETA: 232s - loss: 0.1373 - acc: 0.947 - ETA: 231s - loss: 0.1276 - acc: 0.951 - ETA: 229s - loss: 0.1212 - acc: 0.957 - ETA: 228s - loss: 0.1172 - acc: 0.959 - ETA: 226s - loss: 0.1238 - acc: 0.957 - ETA: 224s - loss: 0.1223 - acc: 0.956 - ETA: 222s - loss: 0.1222 - acc: 0.957 - ETA: 220s - loss: 0.1290 - acc: 0.953 - ETA: 219s - loss: 0.1255 - acc: 0.954 - ETA: 217s - loss: 0.1274 - acc: 0.953 - ETA: 216s - loss: 0.1225 - acc: 0.954 - ETA: 215s - loss: 0.1243 - acc: 0.955 - ETA: 213s - loss: 0.1244 - acc: 0.954 - ETA: 211s - loss: 0.1264 - acc: 0.952 - ETA: 209s - loss: 0.1254 - acc: 0.953 - ETA: 207s - loss: 0.1231 - acc: 0.954 - ETA: 205s - loss: 0.1219 - acc: 0.954 - ETA: 203s - loss: 0.1231 - acc: 0.954 - ETA: 201s - loss: 0.1223 - acc: 0.955 - ETA: 199s - loss: 0.1209 - acc: 0.956 - ETA: 198s - loss: 0.1187 - acc: 0.957 - ETA: 196s - loss: 0.1197 - acc: 0.9

9392/9392 [==============================] - ETA: 224s - loss: 0.0924 - acc: 0.984 - ETA: 222s - loss: 0.0711 - acc: 0.981 - ETA: 220s - loss: 0.0664 - acc: 0.980 - ETA: 221s - loss: 0.0649 - acc: 0.980 - ETA: 220s - loss: 0.0695 - acc: 0.978 - ETA: 218s - loss: 0.0687 - acc: 0.977 - ETA: 217s - loss: 0.0663 - acc: 0.977 - ETA: 215s - loss: 0.0681 - acc: 0.977 - ETA: 214s - loss: 0.0652 - acc: 0.980 - ETA: 212s - loss: 0.0633 - acc: 0.981 - ETA: 211s - loss: 0.0661 - acc: 0.979 - ETA: 209s - loss: 0.0626 - acc: 0.980 - ETA: 209s - loss: 0.0599 - acc: 0.981 - ETA: 208s - loss: 0.0578 - acc: 0.982 - ETA: 207s - loss: 0.0565 - acc: 0.982 - ETA: 205s - loss: 0.0549 - acc: 0.982 - ETA: 203s - loss: 0.0548 - acc: 0.983 - ETA: 202s - loss: 0.0564 - acc: 0.982 - ETA: 201s - loss: 0.0570 - acc: 0.981 - ETA: 199s - loss: 0.0591 - acc: 0.980 - ETA: 198s - loss: 0.0571 - acc: 0.981 - ETA: 197s - loss: 0.0564 - acc: 0.981 - ETA: 196s - loss: 0.0581 - acc: 0.979 - ETA: 194s - loss: 0.0566 - acc: 0.9

9392/9392 [==============================] - ETA: 275s - loss: 0.0291 - acc: 0.994 - ETA: 270s - loss: 0.0240 - acc: 0.997 - ETA: 264s - loss: 0.0240 - acc: 0.994 - ETA: 258s - loss: 0.0323 - acc: 0.988 - ETA: 250s - loss: 0.0308 - acc: 0.989 - ETA: 244s - loss: 0.0399 - acc: 0.987 - ETA: 239s - loss: 0.0372 - acc: 0.987 - ETA: 236s - loss: 0.0393 - acc: 0.987 - ETA: 233s - loss: 0.0406 - acc: 0.987 - ETA: 234s - loss: 0.0394 - acc: 0.988 - ETA: 233s - loss: 0.0412 - acc: 0.987 - ETA: 231s - loss: 0.0398 - acc: 0.987 - ETA: 229s - loss: 0.0377 - acc: 0.988 - ETA: 228s - loss: 0.0377 - acc: 0.988 - ETA: 228s - loss: 0.0375 - acc: 0.988 - ETA: 228s - loss: 0.0400 - acc: 0.988 - ETA: 228s - loss: 0.0387 - acc: 0.988 - ETA: 225s - loss: 0.0382 - acc: 0.988 - ETA: 222s - loss: 0.0371 - acc: 0.989 - ETA: 219s - loss: 0.0360 - acc: 0.989 - ETA: 217s - loss: 0.0357 - acc: 0.989 - ETA: 214s - loss: 0.0364 - acc: 0.988 - ETA: 212s - loss: 0.0358 - acc: 0.989 - ETA: 209s - loss: 0.0346 - acc: 0.9

In [21]:
y_bank_pred = model.predict_classes(X_bank_test) - 1
accuracy_score(y_bank_pred, test_bank_labels)

3313/3313 [==============================] - ETA: 69 - ETA: 45 - ETA: 37 - ETA: 33 - ETA: 30 - ETA: 29 - ETA: 27 - ETA: 26 - ETA: 25 - ETA: 24 - ETA: 24 - ETA: 24 - ETA: 23 - ETA: 23 - ETA: 22 - ETA: 22 - ETA: 22 - ETA: 21 - ETA: 21 - ETA: 21 - ETA: 20 - ETA: 20 - ETA: 20 - ETA: 19 - ETA: 19 - ETA: 19 - ETA: 18 - ETA: 18 - ETA: 18 - ETA: 18 - ETA: 17 - ETA: 17 - ETA: 17 - ETA: 17 - ETA: 16 - ETA: 16 - ETA: 16 - ETA: 16 - ETA: 16 - ETA: 15 - ETA: 15 - ETA: 15 - ETA: 14 - ETA: 14 - ETA: 14 - ETA: 14 - ETA: 13 - ETA: 13 - ETA: 13 - ETA: 13 - ETA: 13 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 9 - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 24s 

0.685481436764262